## Audio Analysis of WAV files

Here we are looking for best way to reppresent audio recordings in a spectrogram for deeplearning

In [1]:
# load stuff we need
import json
import logging
import os
import time
import warnings
import librosa
import librosa.display # now seperate
import numpy as np
import pandas as pd
import pydub
import sklearn.preprocessing

from tqdm import tqdm
import h5py as h5py
from config import *

import random
import soundfile as sf

from datetime import datetime

C:\data\programs\Anaconda3\envs\tf-gpu-sensing-clues\lib\site-packages\pydub\utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
# Project directory
project_name     = 'amsterdam_custom_samples'

# Other settings
proportion_train = 0.5

In [3]:
# Set your serval data folder (should be correctly set already)
serval_data_folder = "../data"
project_data_folder = serval_data_folder + '/' + project_name

In [4]:
# Set input files
input_wav_samples_to_combine_filepath     = project_data_folder + "/csv_files/input_wav_samples_to_combine.csv"

# Both input and target
target_resampled_wav_samples              = project_data_folder + "/csv_files/output_resampled_wav_samples.csv"
target_resampled_wav_samples_backup       = project_data_folder + "/csv_files/output_resampled_wav_samples_backup_" + datetime.today().strftime('%Y%m%d_%H%M%S') + '.csv'

# Set output filepath
target_resampled_wav_folder               = project_data_folder + "/wav_samples"

In [5]:
# Load selected wav samples from csv
df_combinations = pd.read_csv(input_wav_samples_to_combine_filepath, sep=";")

# Load all available wav samples from csv
df_samples = pd.read_csv(target_resampled_wav_samples, sep=";")

# Write backup csv back to disk
df_samples.to_csv(target_resampled_wav_samples_backup, sep=";", index=False)

# Print
display(df_combinations)

,file_1_label_filter,file_1_filepath_filter,file_2_label_filter,file_2_filepath_filter,cnt_generated_samples,resample_project_name
0,1000,0db,KEEP_ALL,6db,100,custom_youtube_1000
1,1001,0db,KEEP_ALL,6db,100,custom_youtube_1001
2,1002,0db,KEEP_ALL,6db,100,custom_youtube_1002
3,1003,0db,KEEP_ALL,6db,100,custom_youtube_1003
4,1004,0db,KEEP_ALL,6db,100,custom_youtube_1004
5,1005,0db,KEEP_ALL,6db,100,custom_youtube_1005
6,1006,0db,KEEP_ALL,6db,100,custom_youtube_1006
7,1007,0db,KEEP_ALL,6db,100,custom_youtube_1007
8,1008,0db,KEEP_ALL,6db,100,custom_youtube_1008
9,1009,0db,KEEP_ALL,6db,100,custom_youtube_1009


In [6]:
# RESAMPLE 16 bit 16 kHz mono
import numpy as np
import scipy.signal
import librosa
import resampy

def resample(file):
    # Load in some audio
    x, sr_orig = librosa.load(file, sr=None, mono=True)

    sr_new = 16000
    # Or use the pre-built high-quality filter
    y = resampy.resample(x, sr_orig, sr_new, filter='kaiser_best') 
    return(y,sr_new)

In [7]:
def combine_samples(df_samples, train_or_test):
    df_samples_target = df_samples.copy(deep=True)

    for index, row in df_combinations.iterrows():
        ## Get and create output folder if it does not exist yet
        target_filepath = target_resampled_wav_folder + '/' + train_or_test + '/combine_' + row.resample_project_name
        if not os.path.exists(os.path.dirname(target_filepath + '/')):
            os.makedirs(os.path.dirname(target_filepath + '/'))

        ## Get selected samples
        df_filtered_samples_1 = df_samples.loc[(df_samples.label.isin([row.file_1_label_filter])    | (row.file_1_label_filter in ['*', 'KEEP_ALL'])) &
                                               (df_samples.filepath.apply(lambda x : row.file_1_filepath_filter in x) | (row.file_1_filepath_filter in ['*', 'KEEP_ALL'])) &
                                               (df_samples.ind_train == train_or_test)]
        df_filtered_samples_2 = df_samples.loc[(df_samples.label.isin([row.file_2_label_filter])    | (row.file_2_label_filter in ['*', 'KEEP_ALL'])) &
                                               (df_samples.filepath.apply(lambda x : row.file_2_filepath_filter in x) | (row.file_2_filepath_filter in ['*', 'KEEP_ALL'])) &
                                               (df_samples.ind_train == train_or_test)]

        if (len(df_filtered_samples_1) <= 3) or (len(df_filtered_samples_2) <= 3):
            print('[WARNING]: Invalid filter used; found samples 1: ', len(df_filtered_samples_1), '; found samples 2: ', len(df_filtered_samples_2), sep='')
            print(row)
            continue

        # Generate random sample
        permutation_1 = np.random.choice(range(0, len(df_filtered_samples_1)), size=row.cnt_generated_samples, replace=True)
        permutation_2 = np.random.choice(range(0, len(df_filtered_samples_2)), size=row.cnt_generated_samples, replace=True)

        ## Iterate over all samples
        for index in tqdm(range(1, row.cnt_generated_samples)):
            file1 = df_filtered_samples_1.iloc[permutation_1[index]].filepath
            file2 = df_filtered_samples_2.iloc[permutation_2[index]].filepath

            filename1 = os.path.basename(os.path.splitext(os.path.normpath(file1))[0])
            filename2 = os.path.basename(os.path.splitext(os.path.normpath(file2))[0])

            target_combined_filepath = target_filepath + '/' + filename1 + '__' + filename2 + '.wav'

            df_wav_samples_file1 = df_samples.loc[df_samples.filepath.apply(lambda x : file1 in x)]
            df_wav_samples_file2 = df_samples.loc[df_samples.filepath.apply(lambda x : file2 in x)]

            df = df_wav_samples_file1.copy(deep=True).append(df_wav_samples_file2, ignore_index=True)
            df.source = 'combined/' + df.source
            df.filename = target_combined_filepath
            df.filepath = target_combined_filepath
            df.filename_old = filename1 + '//' + filename2
            df.filepath_old = ""

            df = df.drop_duplicates(subset = ["label"])

            if (len(df_wav_samples_file1) == 0) or (len(df_wav_samples_file2) == 0):
                print('[ERROR]: Invalid file selected; file1: ', file1, '; file2: ', file2, sep='')
                continue

            df_samples_target.append(df, ignore_index=True)

            y1, sr1 = resample(file1)
            y2, sr2 = resample(file2)
            if len(y1) > len(y2):
                y2 = y2 + y1[0:len(y2)]
                sf.write(target_combined_filepath, y2, sr2, subtype='PCM_16')
            else:
                y1 = y1 + y2[0:len(y1)]
                sf.write(target_combined_filepath, y1, sr1, subtype='PCM_16')

    return(df_samples_target)

In [8]:
df_samples_target_train = combine_samples(df_samples, "train")

100%|██████████| 99/99 [00:31<00:00,  3.13it/s]


In [9]:
df_samples_target_train_eval = combine_samples(df_samples_target_train, "eval")

100%|██████████| 99/99 [00:30<00:00,  3.25it/s]


In [10]:
# Write full resampled wav files back to disk
df_samples_target_train_eval.to_csv(target_resampled_wav_samples, sep=';', index=False)